## Class-Dependent Service Analysis

This example demonstrates **class-dependent service allocation**, which is an advanced load-dependent modeling technique.

### Concept:
- **Class1 jobs**: Can utilize up to 2 servers based on their population at the queue
- **Class2 jobs**: Always receive single-server treatment regardless of population
- **Service allocation**: min(Class1_population, 2) servers allocated to Class1

### Key Features:

1. **Asymmetric Service**: Different job classes receive different levels of service capacity
2. **Dynamic Allocation**: Server allocation changes based on specific class populations
3. **Resource Prioritization**: Class1 gets priority access to multiple servers

### Implementation Comparison:

**MATLAB:**
```matlab
node{2}.setClassDependence(@(ni) min(ni(1),c))
```

**Python:**
```python
node2.set_class_dependence(lambda ni: min(ni[0, 0], c))
```

**Java:**
```java
node2.setClassDependence(ni -> Math.min(ni.get(0, 0), c));
```

### Matrix Indexing Notes:
- **MATLAB**: 1-indexed, so `ni(1)` is class 1, `ni(2)` is class 2
- **Python/Java**: 0-indexed, so `ni[0, 0]` is class 1, `ni[0, 1]` is class 2
- The Matrix has shape (1, R) where R is the number of job classes

### Expected Behavior:
- Class1 should have better performance (lower response times) due to multi-server access
- Class2 performance is limited by single-server constraint
- Total system performance depends on the interaction between both classes

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
# Model parameters
N = 16  # number of jobs in Class1
c = 2   # number of servers available for Class1

# Create class-dependent model
cdmodel = Network('cdmodel')
node1 = Delay(cdmodel, 'Delay')
node2 = Queue(cdmodel, 'Queue1', SchedStrategy.PS)

# Create job classes
jobclass1 = ClosedClass(cdmodel, 'Class1', N, node1, 0)
jobclass2 = ClosedClass(cdmodel, 'Class2', N // 2, node1, 0)  # N/2 = 8 jobs

# Set service times
node1.set_service(jobclass1, Exp.fit_mean(1.0))  # Class1: mean = 1.0
node1.set_service(jobclass2, Exp.fit_mean(2.0))  # Class2: mean = 2.0
node2.set_service(jobclass1, Exp.fit_mean(1.5))  # Class1: mean = 1.5
node2.set_service(jobclass2, Exp.fit_mean(2.5))  # Class2: mean = 2.5

# Create routing matrix
P = cdmodel.init_routing_matrix()
P.set(jobclass1, jobclass1, node1, node2, 1.0)
P.set(jobclass1, jobclass1, node2, node1, 1.0)
P.set(jobclass2, jobclass2, node1, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node1, 1.0)
cdmodel.link(P)

In [ ]:
# Set class dependence function
# In MATLAB: @(ni) min(ni(1),c) where ni(1) is Class1 population at the station
# In Python: lambda ni: min(ni[0, 0], c) where ni[0, 0] is Class1 population
print("Setting class dependence...")

beta = lambda ni: min(ni[0, 0], c)
node2.set_class_dependence(beta)
print(f"Set class dependence: min(Class1_population, {c})")

In [ ]:
# Solve with MVA using QD (queue decomposition) method
# Note: QD method may not be available for this model in Python
try:
    solver_mva_qd = MVA(cdmodel, method='qd')
    cdAvgTableCD = solver_mva_qd.avg_table()
    print("MVA QD Results:")
    print(cdAvgTableCD)
except Exception as e:
    print(f"MVA QD solver not available: {e}")
    
    # Try with exact MVA as fallback
    print("\nTrying MVA Exact method as fallback...")
    try:
        solver_mva_exact = MVA(cdmodel)
        cdAvgTableExact = solver_mva_exact.avg_table()
        print("MVA Exact Results:")
        print(cdAvgTableExact)
    except Exception as e2:
        print(f"MVA also failed: {e2}")

In [ ]:
# Solve with CTMC (exact solution)try:    solver_ctmc = CTMC(cdmodel)    cdAvgTableCTMC = solver_ctmc.avg_table()    print("CTMC Results:")    print(cdAvgTableCTMC)except Exception as e:    print(f"CTMC solver not available or failed: {e}")